In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
# key => 
# 산업안전보건법 + "1"
# 건설기술진흥법 + "2"
# 위험물안전관리법 + '3'
# 전기안전관리법 + "4"
# 화학물질관리법 + "5"

In [2]:
df_industrial = pd.read_excel("산업안전보건법.xlsx")
df_erection = pd.read_excel("건설기술진흥법.xlsx")
df_danger = pd.read_excel("위험물안전관리법.xlsx")
df_electric = pd.read_excel("전기안전관리법.xlsx")
df_chemistry = pd.read_excel("화학물질관리법.xlsx")
df_info = pd.DataFrame()
df_info = df_info.append(df_industrial,ignore_index = True)
df_info = df_info.append(df_erection,ignore_index = True)
df_info = df_info.append(df_danger,ignore_index = True)
df_info = df_info.append(df_electric,ignore_index = True)
df_info = df_info.append(df_chemistry,ignore_index = True)
df_info

,키,조내용
0,10012,제1조(목적) 이 법은 산업 안전 및 보건에 관한 기준을 확립하고 그 책임의 소재를...
1,20012,제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2020.5.2...
2,30012,"제3조(적용 범위) 이 법은 모든 사업에 적용한다. 다만, 유해ㆍ위험의 정도, 사업..."
3,40012,제4조(정부의 책무) 정부는 이 법의 목적을 달성하기 위하여 다음 각 호의 사항을 ...
4,50012,제5조(사업주 등의 의무) 사업주(제77조에 따른 특수형태근로종사자로부터 노무를 제...
...,...,...
452,600015,제60조(벌칙) 제43조제2항에 따라 즉시 신고를 하지 아니한 자는 2년 이하의 징...
453,610015,제61조(벌칙) 다음 각 호의 어느 하나에 해당하는 자는 1년 이하의 징역 또는 3...
454,620015,제62조(벌칙) 다음 각 호의 어느 하나에 해당하는 자는 6개월 이하의 징역 또는 ...
455,630015,"제63조(양벌규정) 법인의 대표자나 법인 또는 개인의 대리인, 사용인, 그 밖의 종..."


In [3]:
#df_info.to_excel('전체법.xlsx',index = False)

In [4]:
# 불용어 리스트
# 불용어, 불필요 단어 제거
stop_words_df = pd.read_excel("stopwords.xlsx")
stop_words_df

,불용어
0,아
1,휴
2,아이구
3,아이쿠
4,아이고
...,...
694,따른
695,필요한
696,등을
697,해당하는


In [5]:
posts = df_info.get("조내용")
posts_key = df_info.get("키")
key = [i for i in posts_key]
posts

0      제1조(목적) 이 법은 산업 안전 및 보건에 관한 기준을 확립하고 그 책임의 소재를...
1      제2조(정의) 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2020.5.2...
2      제3조(적용 범위) 이 법은 모든 사업에 적용한다. 다만, 유해ㆍ위험의 정도, 사업...
3      제4조(정부의 책무) 정부는 이 법의 목적을 달성하기 위하여 다음 각 호의 사항을 ...
4      제5조(사업주 등의 의무) 사업주(제77조에 따른 특수형태근로종사자로부터 노무를 제...
                             ...                        
452    제60조(벌칙) 제43조제2항에 따라 즉시 신고를 하지 아니한 자는 2년 이하의 징...
453    제61조(벌칙) 다음 각 호의 어느 하나에 해당하는 자는 1년 이하의 징역 또는 3...
454    제62조(벌칙) 다음 각 호의 어느 하나에 해당하는 자는 6개월 이하의 징역 또는 ...
455    제63조(양벌규정) 법인의 대표자나 법인 또는 개인의 대리인, 사용인, 그 밖의 종...
456    제64조(과태료) 다음 각 호의 어느 하나에 해당하는 자에게는 1천만원 이하의 과태...
Name: 조내용, Length: 457, dtype: object

In [6]:
def get_key(**key) :
    dic = {}
    for k, y in key.items():
        if k[-1] == "1" :
            s = "OSH"+k[0:(len(k)-4)]
        elif k[-1]=="2":
            s = "CTP"+k[0:(len(k)-4)]
        elif k[-1]=="3":
            s = "HSM"+k[0:(len(k)-4)]
        elif k[-1]=="4":
            s = "ESM"+k[0:(len(k)-4)]
        elif k[-1]=="5":
            s = "CMM"+k[0:(len(k)-4)]     
        if k[-3] != "0":
            s += "의 "+k[-3]   
        dic[s] = y
    return dic   

In [7]:
def get_law(law):
    if law == 'OSH':
        num = '1'
    elif law == 'CTP':
        num = '2'
    elif law == 'HSM':
        num = '3'
    elif law == 'ESM':
        num = '4'
    elif law == 'CMM':
        num = '5'
    return num

In [8]:
from konlpy.tag import Okt
okt = Okt()

oo = okt.pos(posts[0],
        norm=True,   # 정규화(normalization)
        stem=True    # 어간추출(stemming)
        )
print(oo)

[('제', 'Noun'), ('1조', 'Number'), ('(', 'Punctuation'), ('목적', 'Noun'), (')', 'Punctuation'), ('이', 'Noun'), ('법', 'Noun'), ('은', 'Josa'), ('산업', 'Noun'), ('안전', 'Noun'), ('및', 'Noun'), ('보건', 'Noun'), ('에', 'Josa'), ('관', 'Noun'), ('한', 'Josa'), ('기준', 'Noun'), ('을', 'Josa'), ('확립', 'Noun'), ('하고', 'Josa'), ('그', 'Noun'), ('책임', 'Noun'), ('의', 'Josa'), ('소재', 'Noun'), ('를', 'Josa'), ('명확하다', 'Adjective'), ('하다', 'Verb'), ('산업', 'Noun'), ('재해', 'Noun'), ('를', 'Josa'), ('예방', 'Noun'), ('하고', 'Josa'), ('쾌적하다', 'Adjective'), ('작업', 'Noun'), ('환경', 'Noun'), ('을', 'Josa'), ('조', 'Modifier'), ('성함', 'Noun'), ('으로써', 'Josa'), ('노무', 'Noun'), ('를', 'Josa'), ('제공', 'Noun'), ('하다', 'Verb'), ('사람', 'Noun'), ('의', 'Josa'), ('안전', 'Noun'), ('및', 'Noun'), ('보건', 'Noun'), ('을', 'Josa'), ('유지', 'Noun'), ('ㆍ', 'Foreign'), ('증진', 'Noun'), ('함', 'Noun'), ('을', 'Josa'), ('목적', 'Noun'), ('으로', 'Josa'), ('하다', 'Verb'), ('.', 'Punctuation'), ('<', 'Punctuation'), ('개정', 'Noun'), ('2020.5', 'Number'), ('.', '

In [9]:
def tokenizer_1(raw_texts, pos=["Noun","Alpha","Verb","Number"], stopword=list(stop_words_df.get("불용어"))):
    p = okt.pos(raw_texts, 
            norm=True,   # 정규화(normalization)
            stem=True    # 어간추출(stemming)
            )
    o = [word for word, tag in p if len(word) > 1 and tag in pos and word[0] not in stopword]
    return(o)

tokenizer_1(posts[80])


['81조',
 '기계',
 '기구',
 '대통령령',
 '기계',
 '기구',
 '설비',
 '건축물',
 '타인',
 '대여',
 '거나',
 '대여받다',
 '안전',
 '보건']

In [10]:
from konlpy.tag import Mecab

In [11]:
tagger = Mecab('C:/mecab/mecab-ko-dic')
# from eunjeon import Mecab
# tagger = Mecab()
# 윈도우에서 Mecab 쓰려면 Mecab 폴더를 C드라이브에 만든 후 라이브러리 설치한 다음에 경로지정을 해주어야 함


def tokenizer_2(raw_texts, pos=["Noun","Alpha","Verb","Number"], stop_words=list(stop_words_df.get("불용어"))):
    nouns = []
  
    for noun in tagger.nouns(raw_texts):
        if noun not in stop_words:
            nouns.append(noun)
    return nouns

tokenizer_2(posts[80])

['기계', '대여자', '조치', '기계', '건축물', '대여', '필요', '안전', '조치', '보건', '조치']

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorize = TfidfVectorizer(
    tokenizer=tokenizer_1, # 문장에 대한 tokenizer (위에 정의한 함수 이용)
    min_df=1,            # 단어가 출현하는 최소 문서의 개수
    sublinear_tf=True    # tf값에 1+log(tf)를 적용하여 tf값이 무한정 커지는 것을 막음
)

X = vectorize.fit_transform(posts)
X.toarray() 

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [13]:
# 모든 조에 해당하는 euclidean distance 를 계산
euclidean_result = []
for x in range(len(df_info)):
    tmp = []
    for y in range(len(df_info)):
        tmp.append(euclidean_distances(X[x], X[y])[0][0])
    euclidean_result.append(tmp)
# euclidean_result

In [14]:
# 모든 조에 해당하는 euclidean distance 결과 값을
# DataFrame 형태로 변환
euclidean_result_df = pd.DataFrame(euclidean_result)
euclidean_result_df

,0,1,2,3,4,5,6,7,8,9,...,447,448,449,450,451,452,453,454,455,456
0,0.000000,1.287018,1.414214,1.165998,1.254766,1.329572,1.345013,1.365101,1.306567,1.338770,...,1.352337,1.372577,1.414214,1.404163,1.386000,1.414214,1.409765,1.409174,1.414214,1.409252
1,1.287018,0.000000,1.297864,1.330648,1.230558,1.270694,1.315033,1.335517,1.343985,1.210560,...,1.371830,1.376658,1.401742,1.382693,1.379936,1.412670,1.381800,1.391088,1.384141,1.374472
2,1.414214,1.297864,0.000000,1.414214,1.377286,1.382473,1.410350,1.400083,1.407667,1.381674,...,1.403273,1.286184,1.414214,1.394130,1.397979,1.414214,1.399753,1.404557,1.397930,1.393891
3,1.165998,1.330648,1.414214,0.000000,1.310313,1.244209,1.333259,1.291583,1.247382,1.355435,...,1.252486,1.300174,1.414214,1.400123,1.375494,1.410904,1.396837,1.403805,1.391467,1.397268
4,1.254766,1.230558,1.377286,1.310313,0.000000,1.174423,1.358981,1.350860,1.334742,1.316628,...,1.376101,1.387900,1.414214,1.358628,1.346798,1.408668,1.361305,1.368473,1.396279,1.392412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,1.414214,1.412670,1.414214,1.410904,1.408668,1.403512,1.408280,1.408634,1.407063,1.408073,...,1.404831,1.374507,1.311241,1.296584,1.357984,0.000000,1.288660,1.328792,1.414214,1.368187
453,1.409765,1.381800,1.399753,1.396837,1.361305,1.391531,1.365614,1.384728,1.406121,1.397338,...,1.400022,1.389328,1.343239,1.028503,1.226607,1.288660,0.000000,1.049590,1.403860,1.065491
454,1.409174,1.391088,1.404557,1.403805,1.368473,1.406960,1.391645,1.387743,1.409365,1.403171,...,1.403820,1.386849,1.373870,1.128706,1.212853,1.328792,1.049590,0.000000,1.363034,1.211996
455,1.414214,1.384141,1.397930,1.391467,1.396279,1.393293,1.411458,1.380973,1.414214,1.409965,...,1.403364,1.393448,1.350949,1.371245,1.344552,1.414214,1.403860,1.363034,0.000000,1.382751


In [15]:
law_jo = 'OSH24'
        
if law_jo[-3] == '의':
    if len(law_jo) == 7 :
        jo_key = law_jo[3] + '0' + law_jo[-1] + '1'+ get_law(law_jo[0:3])
    elif len(law_jo) == 8 :
        jo_key = law_jo[3:5] + '0' + law_jo[-1] + '1'+ get_law(law_jo[0:3])
    elif len(law_jo) == 9 :
        jo_key = law_jo[3:6] + '0' + law_jo[-1] + '1'+ get_law(law_jo[0:3])
else:    
    jo_key = law_jo[3:] + '001' + get_law(law_jo[0:3])

In [16]:
key

[10012,
 20012,
 30012,
 40012,
 50012,
 60012,
 70012,
 80012,
 90012,
 100012,
 110012,
 120012,
 130012,
 140012,
 150012,
 160012,
 170012,
 180012,
 190012,
 200012,
 210012,
 220012,
 230012,
 240012,
 250012,
 260012,
 270012,
 280012,
 290012,
 300012,
 310012,
 320012,
 330012,
 340012,
 350012,
 360012,
 370012,
 380012,
 390012,
 400012,
 410012,
 420012,
 430012,
 440012,
 450012,
 460012,
 470012,
 480012,
 490012,
 500012,
 510012,
 520012,
 530012,
 540012,
 550012,
 560012,
 570012,
 580012,
 590012,
 600012,
 610012,
 620012,
 630012,
 640012,
 650012,
 660012,
 670012,
 680012,
 690012,
 700012,
 710012,
 720012,
 730012,
 740012,
 750012,
 760012,
 770012,
 780012,
 790012,
 800012,
 810012,
 820012,
 830012,
 840012,
 850012,
 860012,
 870012,
 880012,
 890012,
 900012,
 910012,
 920012,
 930012,
 940012,
 950012,
 960012,
 970012,
 980012,
 990012,
 1000012,
 1010012,
 1020012,
 1030012,
 1040012,
 1050012,
 1060012,
 1070012,
 1080012,
 1090012,
 1100012,
 1110012

In [17]:
# euclidean distance 값으로 
# 8조와 가장 비슷한 조를 찾기위한 과정
jo_key = 240013
jo = key.index(int(jo_key))
y_euclidean = euclidean_result_df[jo]
dic = {}
for i in range(len(y_euclidean)):
    dic[str(posts_key[i])] = y_euclidean[i] 
dic = get_key(**dic)
#8조와 비슷한 조 찾기 위함
# euclidean distance 를 dic에 씌워 sorting 함(1부터 10위까지)
sorted(dic.items(), key=lambda x: x[1])[0:15]

[('HSM24', 0.0),
 ('HSM26', 1.0209338082931547),
 ('HSM2', 1.0804840674986318),
 ('HSM36', 1.1462646405829289),
 ('HSM6', 1.1833690023424586),
 ('HSM34의 3', 1.184680680592021),
 ('HSM27', 1.1863740871467308),
 ('HSM5', 1.1952619053212579),
 ('HSM23', 1.2037496688771505),
 ('HSM1', 1.221475416949922),
 ('HSM4', 1.2215938356520495),
 ('HSM25', 1.228016044211915),
 ('HSM15', 1.2298054564341816),
 ('HSM14', 1.2298265548930016),
 ('HSM12', 1.2327961840318429)]

In [18]:
cosine_result = []
for x in range(len(df_info)):
    tmp = []
    for y in range(len(df_info)):
        tmp.append(cosine_similarity(X[x], X[y])[0][0])
    cosine_result.append(tmp)

In [19]:
# 모든 조에 해당하는 cosine similarity 결과 값을
# DataFrame 형태로 변환
cosine_result_df = pd.DataFrame(cosine_result)
cosine_result_df

,0,1,2,3,4,5,6,7,8,9,...,447,448,449,450,451,452,453,454,455,456
0,1.000000,0.171792,0.000000,0.320224,0.212782,0.116119,0.095470,0.068250,0.146441,0.103847,...,0.085593,0.058016,0.000000,0.014163,0.039502,0.000000,0.006282,0.007115,0.000000,0.007004
1,0.171792,1.000000,0.157774,0.114688,0.242864,0.192669,0.135344,0.108197,0.096852,0.267272,...,0.059041,0.052407,0.017560,0.044080,0.047889,0.002181,0.045315,0.032437,0.042077,0.055414
2,0.000000,0.157774,1.000000,0.000000,0.051542,0.044384,0.005457,0.019884,0.009236,0.045489,...,0.015413,0.172865,0.000000,0.028201,0.022828,0.000000,0.020346,0.013610,0.022896,0.028534
3,0.320224,0.114688,0.000000,1.000000,0.141540,0.225972,0.111210,0.165906,0.222020,0.081398,...,0.215639,0.154774,0.000000,0.019828,0.054009,0.004676,0.024424,0.014666,0.031910,0.023821
4,0.212782,0.242864,0.051542,0.141540,1.000000,0.310365,0.076585,0.087589,0.109232,0.133245,...,0.053173,0.036866,0.000000,0.077065,0.093068,0.007827,0.073424,0.063641,0.025203,0.030594
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,0.000000,0.002181,0.000000,0.004676,0.007827,0.015077,0.008373,0.007875,0.010086,0.008666,...,0.013224,0.055365,0.140323,0.159435,0.077940,1.000000,0.169678,0.117156,0.000000,0.064032
453,0.006282,0.045315,0.020346,0.024424,0.073424,0.031821,0.067550,0.041265,0.011412,0.023723,...,0.019969,0.034884,0.097854,0.471091,0.247718,0.169678,1.000000,0.449180,0.014588,0.432364
454,0.007115,0.032437,0.013610,0.014666,0.063641,0.010232,0.031663,0.037084,0.006845,0.015555,...,0.014645,0.038325,0.056240,0.363011,0.264494,0.117156,0.449180,1.000000,0.071069,0.265533
455,0.000000,0.042077,0.022896,0.031910,0.025203,0.029367,0.003893,0.046457,0.000000,0.005999,...,0.015284,0.029151,0.087468,0.059843,0.096089,0.000000,0.014588,0.071069,1.000000,0.044000


In [20]:
# cosine similarity 값으로 
# 8조와 가장 비슷한 조를 찾기위한 과정
y_cosine = cosine_result_df[jo]
dic ={}
for i in range(len(y_cosine)):
    dic[str(posts_key[i])] = y_cosine[i]
dic = get_key(**dic)
#8조와 비슷한 조 찾기 위함
# euclidean distance 를 dic에 씌워 sorting 함(1부터 10위까지)
sorted(dic.items(), key=lambda x: x[1], reverse = True)[0:10]

[('HSM24', 1.0),
 ('HSM26', 0.47884707954201805),
 ('HSM2', 0.4162770899408059),
 ('HSM36', 0.3430386868746445),
 ('HSM6', 0.29981890214750717),
 ('HSM34의 3', 0.2982658425160132),
 ('HSM27', 0.2962582626733809),
 ('HSM5', 0.2856744888438981),
 ('HSM23', 0.2754933673390754),
 ('HSM1', 0.253998902893507)]

In [21]:
import openpyxl
# 워크북 열기
wb = openpyxl.load_workbook('TF-IDF결과.xlsx')
# 시트 열기(활성화)
sheet = wb['안전보건법령 간']

# 제시된 틀 만들기
sheet['A1'] = '안전보건법령 간 TF-IDF 기준'
for i in range(10) :
    sheet.cell(row=2, column=i+2).value = i+1
for i in range(0,141) :
    s = "OSH"+str(i+1)
    sheet.cell(row=i+3, column=1).value = s

# 상위 유사조항 10개   
for j in range(0,141) :
    y_cosine = cosine_result_df[j]
    dic ={}
    for i in range(len(y_cosine)):
        dic[str(posts_key[i])] = y_cosine[i]
    dic = get_key(**dic)
    a = sorted(dic.items(), key=lambda x: x[1], reverse = True)[1:11]
    for i in range(10) :
        sheet.cell(row=j+3, column=i+2).value = str(a[i][0])

# 저장
wb.save('TF-IDF결과.xlsx')

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
countv = CountVectorizer(stop_words = list(stop_words_df.get("불용어")))
sp_mat = countv.fit_transform(posts)
# 희소행렬을 np array로
sp_mat.toarray()

C:\Users\assay\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['가서', '같은', '것과', '결과에', '결론을', '관계가', '관련이', '그런', '그럼에도', '그렇게', '그에', '그치지', '김에', '까닭에', '낫다', '년도', '논하지', '누가', '다시', '달려', '대로', '대해', '되는', '되다', '되어', '들면', '들자면', '듯하다', '따르는', '따름이다', '따지지', '때가', '만은', '만이', '많은', '말하면', '말할것도', '몰라도', '몰랏다', '못하다', '못하다하기보다는', '미치다', '바꾸어서', '바꿔', '방면으로', '보면', '보아', '부류의', '비길수', '비추어', '뿐만', '사람들', '상대적으로', '생각이다', '서술한바와같이', '쓰여', '아니다', '아니라', '안다', '안된다', '않고', '않기', '않는다면', '않다', '않다면', '않도록', '않으면', '알겠는가', '어쩔수', '없고', '없다', '예를', '외에', '요만한', '우에', '위에서', '이렇게', '이로', '이르다', '이와', '이유는', '인하여', '임에', '점에서', '정도에', '정도의', '종합한것과같이', '주저하지', '줄은', '지경이다', '틀림없다', '편이', '하고', '하기', '하기만', '하는', '하는것만', '하는것이', '하면', '하지', '한하다', '할수록', '함으로써', '해도', '해서는', '형식으로', '힘이'] not in stop_words.
  warnings.warn('Your stop_words may be inconsis

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [23]:
vectorize = CountVectorizer(
    tokenizer=tokenizer_2, # 문장에 대한 tokenizer (위에 정의한 함수 이용)
    min_df=1,            # 단어가 출현하는 최소 문서의 개수
)

X = vectorize.fit_transform(posts)
X.toarray() 

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [24]:
# 모든 조에 해당하는 euclidean distance 를 계산
euclidean_result = []
for x in range(len(df_info)):
    tmp = []
    for y in range(len(df_info)):
        tmp.append(euclidean_distances(X[x], X[y])[0][0])
    euclidean_result.append(tmp)
# euclidean_result

In [25]:
# 모든 조에 해당하는 euclidean distance 결과 값을
# DataFrame 형태로 변환
euclidean_result_df = pd.DataFrame(euclidean_result)
euclidean_result_df

,0,1,2,3,4,5,6,7,8,9,...,447,448,449,450,451,452,453,454,455,456
0,0.000000,25.039968,8.717798,14.000000,11.135529,7.071068,9.486833,24.372115,12.083046,16.155494,...,8.246211,6.782330,6.480741,20.049938,23.832751,6.164414,14.696938,11.445523,10.000000,33.941125
1,25.039968,0.000000,24.351591,28.372522,24.103942,24.839485,26.172505,33.511192,27.147744,27.166155,...,25.980762,25.670995,25.865034,31.543621,34.073450,25.748786,28.687977,27.276363,26.438608,41.218928
2,8.717798,24.351591,0.000000,17.320508,12.409674,8.602325,10.954451,24.617067,13.638182,16.522712,...,9.055385,7.874008,8.124038,20.542639,24.372115,7.745967,15.491933,12.369317,11.224972,34.117444
3,14.000000,28.372522,17.320508,0.000000,16.552945,15.811388,17.606817,26.608269,17.832555,21.840330,...,16.792856,16.792856,16.613248,24.899799,27.712813,16.492423,21.071308,19.104973,18.275667,37.094474
4,11.135529,24.103942,12.409674,16.552945,0.000000,9.899495,13.711309,24.899799,14.832397,17.691806,...,12.727922,12.806248,12.328828,22.045408,25.219040,12.165525,17.492856,15.329710,14.628739,35.270384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,6.164414,25.748786,7.745967,16.492423,12.165525,6.928203,8.717798,24.248711,12.328828,15.716234,...,8.000000,5.291503,3.464102,19.131126,23.065125,0.000000,13.038405,9.746794,9.165151,32.893768
453,14.696938,28.687977,15.491933,21.071308,17.492856,14.899664,15.362291,27.276363,17.776389,20.420578,...,15.684387,14.212670,13.638182,14.491377,19.849433,13.038405,0.000000,10.148892,16.370706,24.738634
454,11.445523,27.276363,12.369317,19.104973,15.329710,11.958261,12.609520,25.903668,15.716234,18.493242,...,12.529964,10.908712,10.344080,15.968719,20.371549,9.746794,10.148892,0.000000,13.076697,29.478806
455,10.000000,26.438608,11.224972,18.275667,14.628739,10.488088,11.832160,25.534291,14.560220,17.748239,...,11.045361,9.486833,9.486833,20.639767,24.248711,9.165151,16.370706,13.076697,0.000000,34.234486


In [26]:
# euclidean distance 값으로 
# 8조와 가장 비슷한 조를 찾기위한 과정
y_euclidean = euclidean_result_df[jo]
dic = {}
for i in range(len(y_euclidean)):
    dic[str(posts_key[i])] = y_euclidean[i]
dic = get_key(**dic)
#8조와 비슷한 조 찾기 위함
# euclidean distance 를 dic에 씌워 sorting 함(1부터 10위까지)
sorted(dic.items(), key=lambda x: x[1])[0:10]

[('HSM24', 0.0),
 ('HSM1', 7.0710678118654755),
 ('HSM34의 3', 7.0710678118654755),
 ('HSM4', 7.211102550927978),
 ('HSM29', 7.211102550927978),
 ('HSM3', 7.280109889280518),
 ('CTP36', 7.3484692283495345),
 ('CMM41', 7.3484692283495345),
 ('CMM41의 2', 7.3484692283495345),
 ('CMM42', 7.3484692283495345)]

In [27]:
import openpyxl
# 워크북 열기
wb = openpyxl.load_workbook('count결과(Eu).xlsx')
# 시트 열기(활성화)
sheet = wb['안전보건법령 간']

# 제시된 틀 만들기
sheet['A1'] = '안전보건법령 간 count결과(Eu) 기준'
for i in range(10) :
    sheet.cell(row=2, column=i+2).value = i+1
for i in range(0,141) :
    s = "OSH"+str(i+1)
    sheet.cell(row=i+3, column=1).value = s

# 상위 유사조항 10개   
for j in range(0,141) :
    y_euclidean = euclidean_result_df[j]
    dic ={}
    for i in range(len(y_euclidean)):
        dic[str(posts_key[i])] = y_euclidean[i]
    dic = get_key(**dic)
    a = sorted(dic.items(), key=lambda x: x[1])[1:11]
    for i in range(10) :
        sheet.cell(row=j+3, column=i+2).value = str(a[i][0])

# 저장
wb.save('count결과(Eu).xlsx')

In [28]:
cosine_result = []
for x in range(len(df_info)):
    tmp = []
    for y in range(len(df_info)):
        tmp.append(cosine_similarity(X[x], X[y])[0][0])
    cosine_result.append(tmp)

In [29]:
# 모든 조에 해당하는 cosine similarity 결과 값을
# DataFrame 형태로 변환
cosine_result_df = pd.DataFrame(cosine_result)
cosine_result_df

,0,1,2,3,4,5,6,7,8,9,...,447,448,449,450,451,452,453,454,455,456
0,1.000000,0.203186,0.052870,0.567462,0.383571,0.221470,0.047023,0.063764,0.178279,0.061602,...,0.158610,0.041996,0.046676,0.049041,0.057183,0.000000,0.041345,0.018692,0.000000,0.017168
1,0.203186,1.000000,0.306160,0.136919,0.353960,0.237855,0.085990,0.092311,0.095499,0.200266,...,0.085940,0.068264,0.009484,0.043843,0.039836,0.011790,0.036403,0.030384,0.064073,0.049997
2,0.052870,0.306160,1.000000,0.075942,0.250958,0.180656,0.000000,0.085333,0.046271,0.105523,...,0.226415,0.179847,0.066630,0.056004,0.040814,0.082832,0.039346,0.040025,0.000000,0.040844
3,0.567462,0.136919,0.075942,1.000000,0.346898,0.262573,0.082553,0.178091,0.227625,0.062922,...,0.151884,0.026810,0.044697,0.046961,0.067796,0.037043,0.035192,0.017900,0.014380,0.018266
4,0.383571,0.353960,0.250958,0.346898,1.000000,0.573409,0.121747,0.185727,0.230792,0.202025,...,0.205329,0.018122,0.100707,0.093113,0.112803,0.100157,0.101099,0.064529,0.019439,0.039510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,0.000000,0.011790,0.082832,0.037043,0.100157,0.000000,0.036835,0.037462,0.000000,0.077209,...,0.000000,0.131590,0.585018,0.230497,0.217574,1.000000,0.388650,0.322139,0.000000,0.268964
453,0.041345,0.036403,0.039346,0.035192,0.101099,0.058864,0.113734,0.050419,0.060308,0.050429,...,0.009837,0.109388,0.225788,0.675187,0.541073,0.388650,1.000000,0.688597,0.000000,0.760186
454,0.018692,0.030384,0.040025,0.017900,0.064529,0.000000,0.083063,0.036204,0.000000,0.024872,...,0.013342,0.084781,0.188457,0.584115,0.507114,0.322139,0.688597,1.000000,0.045472,0.531415
455,0.000000,0.064073,0.000000,0.014380,0.019439,0.000000,0.000000,0.009695,0.009857,0.007493,...,0.032154,0.051081,0.000000,0.095440,0.099362,0.000000,0.000000,0.045472,1.000000,0.055684


In [38]:
cosine_result_df.values.tolist()[1]

[0.20318563844357893,
 0.999999999999999,
 0.30615971624938426,
 0.13691878750113395,
 0.353959540515352,
 0.23785483713265243,
 0.08598965956896519,
 0.09231062770189838,
 0.09549916347754896,
 0.20026552785330498,
 0.12085723107948844,
 0.1013700498893391,
 0.13618161595789596,
 0.084734268689751,
 0.26568624024940996,
 0.1797254106730326,
 0.18930704150480704,
 0.18663792646814922,
 0.16961507134124557,
 0.1032282599229018,
 0.11626148519619373,
 0.18917599887179815,
 0.16181326784598615,
 0.1617921667614627,
 0.11835534849134417,
 0.1294832843208119,
 0.17044643665264492,
 0.08867748269482187,
 0.25310444884872063,
 0.20523145763752693,
 0.22582821126639088,
 0.13660021038725273,
 0.09715982362539967,
 0.2147249366058269,
 0.28839558133685744,
 0.279085975209623,
 0.2390451746905995,
 0.22957241569155726,
 0.16793912701966657,
 0.19566024442715008,
 0.23672170102916928,
 0.22739753610313446,
 0.16349027896679813,
 0.13333165304583608,
 0.1167153081808362,
 0.11289430777899981,
 0.1

In [30]:
# cosine similarity 값으로 
# 8조와 가장 비슷한 조를 찾기위한 과정
y_cosine = cosine_result_df[jo]
dic ={}
for i in range(len(y_cosine)):
    dic[str(posts_key[i])] = y_cosine[i]
dic = get_key(**dic)
#8조와 비슷한 조 찾기 위함
# euclidean distance 를 dic에 씌워 sorting 함(1부터 10위까지)
sorted(dic.items(), key=lambda x: x[1], reverse = True)[0:10]

[('HSM24', 0.9999999999999999),
 ('HSM26', 0.6210889339625173),
 ('HSM2', 0.5743214653879933),
 ('HSM37', 0.4871314608132852),
 ('HSM5', 0.4850924354612758),
 ('HSM36', 0.4785822117917143),
 ('HSM27', 0.46836358617850943),
 ('HSM22', 0.4359900220788259),
 ('HSM12', 0.4299253181372339),
 ('HSM39', 0.4155301343249455)]

In [31]:
import openpyxl
# 워크북 열기
wb = openpyxl.load_workbook('count결과(C).xlsx')
# 시트 열기(활성화)
sheet = wb['안전보건법령 간']

# 제시된 틀 만들기
sheet['A1'] = '안전보건법령 간 count결과(C) 기준'
for i in range(10) :
    sheet.cell(row=2, column=i+2).value = i+1
for i in range(0,141) :
    s = "OSH"+str(i+1)
    sheet.cell(row=i+3, column=1).value = s

# 상위 유사조항 10개   
for j in range(0,141) :
    y_cosine = cosine_result_df[j]
    dic ={}
    for i in range(len(y_cosine)):
        dic[str(posts_key[i])] = y_cosine[i]
    dic = get_key(**dic)
    a = sorted(dic.items(), key=lambda x: x[1], reverse = True)[1:11]
    for i in range(10) :
        sheet.cell(row=j+3, column=i+2).value = str(a[i][0])

# 저장
wb.save('count결과(C).xlsx')